# NOVA course on deep learning in remote sensing

# Home exercise


# Objective
-	Compare the effect of training a seedling detector on your **own annotated dataset Vs full dataset** (all annotations merged) on the detector’s performance.



# 1. Data

First, let's intall packages and libraries

In [1]:

!pip -q install geopandas
!pip -q install rasterio
!pip -q install folium matplotlib mapclassify

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 16.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.4/16.4 MB 113.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.9/7.9 MB 119.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.3/21.3 MB 18.0 MB/s eta 0:00:00


In [ ]:
import os, glob, json, shutil
import numpy as np
from pathlib import Path
from datetime import datetime, timedelta
import pandas as pd

# geospatial packages
import geopandas as gpd
from osgeo import gdal, ogr, osr
from shapely.geometry import Polygon
import rasterio as rio
import folium

# plotting libraries
import matplotlib.pyplot as plt
from PIL import Image
import cv2

Let's mount Google Drive now

In [ ]:
# mount google drive
from google.colab import drive
drive.mount('/content/drive')

# home directory
my_drive_path = "/content/drive/MyDrive/HOME_EXAM/data"


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## How many images and annotations had each dataset?

Each classmate got a individual set of images for the training of their models and at the end of the course, all annotations and images were shared to compared perfomances.

In [ ]:
# my dataset
my_dataset_dir = my_drive_path+"/annotated_data/train/20"
img_files_training = [f for f in os.listdir(my_dataset_dir+"/train/images") if f.endswith(".tif")]
img_files_val = [f for f in os.listdir(my_dataset_dir+"/val/images") if f.endswith(".tif")]
annotations = gpd.read_file(my_dataset_dir+"/annotations_20.shp")

my_data = np.array(["My dataset", len(annotations), len(img_files_training), len(img_files_val)])


array(['My dataset', '569', '47', '21'], dtype='<U21')

In [ ]:
# full dataset
full_dataset_dir = my_drive_path+"/annotated_data/train/full_data/"
img_files_training = [f for f in os.listdir(full_dataset_dir+"/all_data/train/images") if f.endswith(".tif")]
img_files_val = [f for f in os.listdir(full_dataset_dir+"/all_data/val/images") if f.endswith(".tif")]
annotations = gpd.read_file(full_dataset_dir+"/full_annotations.shp")

full_data = np.array(["Full dataset", len(annotations), len(img_files_training), len(img_files_val)])

array(['Full dataset', '5074', '270', '117'], dtype='<U21')

In [ ]:
from tabulate import tabulate

# Define the data for the table
data = [my_data,full_data]

# Define the headers for the table
headers = ["Dataset", "Number of annotations", "Number of training images", "Number of validation images"]

# Generate the table
table = tabulate(data, headers, tablefmt="grid")

# Print the table
print(table)


+--------------+-------------------------+-----------------------------+-------------------------------+
| Dataset      |   Number of annotations |   Number of training images |   Number of validation images |
+==============+=========================+=============================+===============================+
| My dataset   |                     569 |                          47 |                            21 |
+--------------+-------------------------+-----------------------------+-------------------------------+
| Full dataset |                    5074 |                         270 |                           117 |
+--------------+-------------------------+-----------------------------+-------------------------------+


# 2. Model training

With these data, two set of models were made and trained, one per dataset.

For each dataset, 9 YOLO v8 were made and trained based on the possible combinations of 3 YOLO sizes (nano, medium and large) and 3 image sizes (416, 608 and 800).

The rest of hyperparameters were set the same (epochs = 300) or as default.

In [ ]:
yolo_sizes = ["nano", "medium", "large"]
image_sizes = [416, 608, 800]

matrix = []

for yolo_size in yolo_sizes:
    for image_size in image_sizes:
        combination = [yolo_size, image_size]
        matrix.append(combination)

# Print the table
table = tabulate(matrix, headers=["YOLO_size", "Image_size"], tablefmt="fancy_grid")
print(table)

╒═════════════╤══════════════╕
│ YOLO_size   │   Image_size │
╞═════════════╪══════════════╡
│ nano        │          416 │
├─────────────┼──────────────┤
│ nano        │          608 │
├─────────────┼──────────────┤
│ nano        │          800 │
├─────────────┼──────────────┤
│ medium      │          416 │
├─────────────┼──────────────┤
│ medium      │          608 │
├─────────────┼──────────────┤
│ medium      │          800 │
├─────────────┼──────────────┤
│ large       │          416 │
├─────────────┼──────────────┤
│ large       │          608 │
├─────────────┼──────────────┤
│ large       │          800 │
╘═════════════╧══════════════╛


After training the models, the performance was assess in COMET